# Translation Memory Retrieval

Note: Preprocessing is a separate module and must be done before using this!

In [1]:
import sys
import nltk
import numpy as np

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/khannatanmai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

In [4]:
input_line = input()

#convert input to lowercase
input_line = input_line.lower()

#tokenise
input_tokens = word_tokenize(input_line)

content_words = [word for word in input_tokens if word not in stop_words] #Removing Stopwords

print(content_words)

I request you to remove the drive safely
['request', 'remove', 'drive', 'safely']


## Edit Distance

### Load TM

Now we are dealing with the whole file which has ~800000 sentences in the TM. 

Approach:
We take a chunk of size N bytes and in that, We take each sentence in the TM and check if any of the content words are present in it. If they are, we then calculate edit-distance and store it. This way we save time as we don't have to calculate edit distance for each sentence in the TM.

This helps with huge files and is a scalable option, unlike the last option which would fail above a certain memory.

Once we have a list of edit distances, we take the N lowest, i.e. N best matches and print from the Target TM.

In [5]:
def read_in_chunks(file_object, chunk_size):
    """Lazy function (generator) to read a file piece by piece."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

In [6]:
N = 5 #Top N matches returned
csize = 3000000 #Chunk Size (In Bytes) ~10000 sentences in a chunk(assuming 30 bytes per sentence)

edit_distance_all = []
indices_all = []

add_to_next_chunk = ''

with open('../tm_data/tm_src_pp.txt') as f: #f = sys.stdin
    last_noun = ''
    
    count = 0
    j = 0
    
    for piece in read_in_chunks(f, csize):

        piece = add_to_next_chunk + piece #Adding cut info from last chunk

        i = len(piece) - 1

        while(i >= 0): #looking for the last endline before the chunk ends
            if(piece[i] == '\n'):
                break

            i -= 1

        add_to_next_chunk = piece[i+1:] #Info for next chunk
        
        piece = piece[:i] #Remove part that has gone to next chunk
        piece = piece.rstrip()
        
        #Now, analysing this piece to get content words in Source TM
        
        src_tm = piece.split('\n')
        
        src_tm_words = [] #Content Words in Source TM
        
        for candidate in src_tm:
            words = candidate.split('\t') #Tab Separated TM
            
            src_tm_words.append(words)
            
        #Finding out which of these have the content words and calculating Edit Distance
        
        for candidate in src_tm_words:
            
            #Check if Content Words present in Candidate
            for word in content_words:
                if(word in candidate):
                    count += 1
                    #print(candidate)

                    ed = nltk.edit_distance(content_words, candidate) #Calculate Edit Distance only if content words exist

                    edit_distance_all.append(ed)
                    indices_all.append(j)

                    break

            j += 1
    
    print('Running Edit Distance on ' + str(count) + ' Candidates out of a possible ' + str(j) + '!\n')

Running Edit Distance on 2330 Candidates out of a possible 772820!



## Get Matches

In [7]:
#Get top N results
edit_distance_all = np.array(edit_distance_all)

sorted_indices = np.argsort(edit_distance_all) #Sorts in ascending order and returns the indices of indices_all array
least_N_indices = sorted_indices[:N] #We want least edit distance

#print(least_N_indices)

final_indices = []

for i in least_N_indices:
    final_indices.append(indices_all[i])
    
print(final_indices)

[216241, 678467, 72651, 550626, 680851]


## Retrieval of Target from TM

In [8]:
tgt_tm_array = []

with open('../tm_data/tm_tgt.txt') as tgt_tm:
    line = tgt_tm.readline()
    
    while line:
        tgt_tm_array.append(line)
        line = tgt_tm.readline()
        
for i in final_indices:
    print(i, tgt_tm_array[i])

216241 क्षैतिज विभाजन और स्तर विन्यास के वास्तविक अभिलोपन की लगभग सीमा तक दमन के ऐसे परिप्रेक्ष्य में शैली समाधान नहीं कर सकी .

678467 वही हैं

72651 १३ नवम्बर

550626 कह दो

680851 अल्लाह ही रात और दिन का उलट-फेर करता है।निश्चय ही आँखें रखनेवालों के लिए इसमें एक शिक्षा है

